# Serving a TensorFlow Model as a REST Endpoint with TensorFlow Serving and SageMaker

We need to understand the application and business context to choose between real-time and batch predictions. Are we trying to optimize for latency or throughput? Does the application require our models to scale automatically throughout the day to handle cyclic traffic requirements? Do we plan to compare models in production through A/B tests?

If our application requires low latency, then we should deploy the model as a real-time API to provide super-fast predictions on single prediction requests over HTTPS. We can deploy, scale, and compare our model prediction servers with SageMaker Endpoints.

<img src="img/sagemaker-architecture.png" width="80%" align="left">

In [14]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [15]:
%store -r training_job_name

In [16]:
try:
    training_job_name
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] Please run the notebooks in the previous TRAIN section before you continue.")
    print("+++++++++++++++++++++++++++++++")

[OK]


# Copy the Model to the Notebook

In [17]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

download: s3://sagemaker-us-east-1-211125778552/tensorflow-training-2024-03-03-04-02-13-539/output/model.tar.gz to ./model.tar.gz


In [18]:
!rm -rf ./model/

In [19]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
metrics/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
metrics/evaluation.json
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
metrics/confusion_matrix.png
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/plugins/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/plugins/profile/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/plugins/profile/2024_03_03_04_06_31/
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tensorboard/train/plugins/profile/2024_03_03_04_06_31/ip-10-0-71-63.ec2.internal.tensorflow_stats.pb
tar: Ignoring unknown extended header ke

In [20]:
!saved_model_cli show --all --dir './model/tensorflow/saved_model/0/'

2024-03-06 23:51:25.266756: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-03-06 23:51:25.266789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 64)
        name: serving_defau

In [21]:
!saved_model_cli run --dir './model/tensorflow/saved_model/0/' --tag_set serve --signature_def serving_default \
    --input_exprs 'input_ids=np.zeros((1,64));input_mask=np.zeros((1,64))'

2024-03-06 23:51:38.839214: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2024-03-06 23:51:38.839248: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-06 23:51:40.261757: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-06 23:51:40.261791: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-06 23:51:40.261833: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (default): /proc/driver/nvidia/version does not exist
2024-03-06 23:51:40.262096: I tensorflow/core/platform/cpu_feature_gu

# Show `inference.py`

In [22]:
!pygmentize ./code/inference.py

import json
import subprocess
import sys

subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow==2.3.1"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers==3.5.1"])
# Workaround for https://github.com/huggingface/tokenizers/issues/120 and
#                https://github.com/kaushaltrivedi/fast-bert/issues/174
# subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', 'tokenizers'])

import tensorflow as tf
from transformers import DistilBertTokenizer

classes = [1, 2, 3, 4, 5]

max_seq_length = 64

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


def input_handler(data, context):
    data_str = data.read().decode("utf-8")
    print("data_str: {}".format(data_str))
    print("type data_str: {}".format(type(data_str)))

    jsonlines = data_str.split("\n")
    print("jsonlines: {}".format(jsonlines))
    print("type jsonlines: {}".format(type(jsonlines)))

    transformed_instances = []

    f

# Deploy the Model
This will create a default `EndpointConfig` with a single model.  

The next notebook will demonstrate how to perform more advanced `EndpointConfig` strategies to support canary rollouts and A/B testing.

_Note:  If not using a US-based region, you may need to adapt the container image to your current region using the following table:_

https://docs.aws.amazon.com/deep-learning-containers/latest/devguide/deep-learning-containers-images.html

In [23]:
import time

timestamp = int(time.time())

tensorflow_model_name = "{}-{}-{}".format(training_job_name, "tf", timestamp)

print(tensorflow_model_name)

tensorflow-training-2024-03-03-04-02-13-539-tf-1709769112


In [24]:
from sagemaker.tensorflow.estimator import TensorFlow

estimator = TensorFlow.attach(training_job_name=training_job_name)


2024-03-03 05:12:09 Starting - Preparing the instances for training
2024-03-03 05:12:09 Downloading - Downloading the training image
2024-03-03 05:12:09 Training - Training image download completed. Training in progress.
2024-03-03 05:12:09 Uploading - Uploading generated training model
2024-03-03 05:12:09 Completed - Training job completed


In [31]:
# requires enough disk space for tensorflow, transformers, and bert downloads
instance_type = "ml.c4.xlarge" # changed from "ml.m4.xlarge"

In [32]:
from sagemaker.tensorflow.model import TensorFlowModel

tensorflow_model = TensorFlowModel(
    name=tensorflow_model_name,
    source_dir="code",
    entry_point="inference.py",
    model_data="s3://{}/{}/output/model.tar.gz".format(bucket, training_job_name),
    role=role,
    framework_version="2.3.1"
)

In [33]:
tensorflow_endpoint_name = "{}-{}-{}".format(training_job_name, "tf", timestamp)

print(tensorflow_endpoint_name)

tensorflow-training-2024-03-03-04-02-13-539-tf-1709769112


In [34]:
tensorflow_model.deploy(
    endpoint_name=tensorflow_endpoint_name,
    initial_instance_count=1,  # Should use >=2 for high(er) availability
    instance_type=instance_type,
    wait=False
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [35]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, tensorflow_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_

In [36]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=tensorflow_endpoint_name)

CPU times: user 36.8 ms, sys: 8.06 ms, total: 44.9 ms
Wall time: 3min


# _Wait Until the ^^ Endpoint ^^ is Deployed_

In [37]:
tensorflow_endpoint_arn = sm.describe_endpoint(EndpointName=tensorflow_endpoint_name)["EndpointArn"]
print(tensorflow_endpoint_arn)

arn:aws:sagemaker:us-east-1:211125778552:endpoint/tensorflow-training-2024-03-03-04-02-13-539-tf-1709769112


# Show the Experiment Tracking Lineage

In [38]:
from sagemaker.lineage.visualizer import LineageTableVisualizer

lineage_table_viz = LineageTableVisualizer(sess)
lineage_table_viz_df = lineage_table_viz.show(endpoint_arn=tensorflow_endpoint_arn)
lineage_table_viz_df

,Name/Source,Direction,Type,Association Type,Lineage Type
0,tensorflow-training-2024-03-03-04-02-13-539-tf...,Input,ModelDeployment,AssociatedWith,action


# Test the Deployed Model

In [39]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=tensorflow_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    content_type="application/jsonlines",
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

content_type is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


# Wait for the Endpoint to Settle Down

In [40]:
import time

time.sleep(30)

# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [41]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted star_rating: {}".format(predicted_class))

Predicted star_rating: {'predicted_label': 5}
Predicted star_rating: {'predicted_label': 1}


# Predict the `star_rating` with `review_body` Samples from our TSV's

In [42]:
import csv

df_reviews = pd.read_csv(
    "./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz",
    delimiter="\t",
    quoting=csv.QUOTE_NONE,
    compression="gzip",
)
df_sample_reviews = df_reviews[["review_body", "star_rating"]].sample(n=5)
df_sample_reviews = df_sample_reviews.reset_index()
df_sample_reviews.shape

(5, 3)

In [43]:
import pandas as pd


def predict(review_body):
    inputs = [{"features": [review_body]}]
    predicted_classes = predictor.predict(inputs)
    return predicted_classes[0]["predicted_label"]


df_sample_reviews["predicted_class"] = df_sample_reviews["review_body"].map(predict)
df_sample_reviews.head(5)

,index,review_body,star_rating,predicted_class
0,46615,Very convenient for multi-platform home use! T...,5,5
1,73581,The program is &#34;clunky!&#34; It is difficu...,5,1
2,69890,Just downloaded this version of Turbotax 2013 ...,5,5
3,13705,Nortons products do their normal great job.,5,5
4,87350,Used TT for many years. This is an awesome pro...,2,4


# Save for Next Notebook(s)

In [44]:
%store tensorflow_model_name

Stored 'tensorflow_model_name' (str)


In [45]:
%store tensorflow_endpoint_name

Stored 'tensorflow_endpoint_name' (str)


In [46]:
%store tensorflow_endpoint_arn

Stored 'tensorflow_endpoint_arn' (str)


In [47]:
%store

Stored variables and their in-db values:
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:211125778552:endpoint
autopilot_model_arn                                   -> 'arn:aws:sagemaker:us-east-1:211125778552:model/au
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-211125778552/data/amazon
balance_dataset                                       -> True
balanced_bias_data_jsonlines_s3_uri                   -> 's3://sagemaker-us-east-1-211125778552/bias-detect
balanced_bias_data_s3_uri                             -> 's3://sagemaker-us-east-1-211125778552/bias-detect
best_candidate_tuning_job_name                        -> 0    tensorflow-training-240306-1651-002-2601ee0c

bias_data_s3_uri                                      -> 's3://sagemaker-us-east-1-211125778552/bias-detect
comprehend_endpoint_arn                               -> 'arn:aws:comprehend:us-east-1:211125778552:documen
comprehend_train_s3_uri          

# Release Resources
To save cost, we should delete the endpoint.

In [48]:
# sm.delete_endpoint(
#      EndpointName=tensorflow_endpoint_name
# )

In [49]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [50]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>

# Internal - DO NOT RUN - WILL REMOVE SOON


In [51]:
# %%bash

# # without split:  tensorflow-training-2021-01-27-02-29-07-903-tf-1611724084
# # with split:  tensorflow-training-2021-01-28-01-19-50-987-tf-1611799952

# aws sagemaker-runtime invoke-endpoint \
#     --endpoint-name "tensorflow-training-2021-01-28-01-19-50-987-tf-1611799952" \
#     --content-type application/jsonlines \
#     --accept application/jsonlines \
#     --body $'{"features":["Amazon gift cards are the best"]}\n{"features":["It is the worst"]}' >(cat) 1>/dev/null

In [52]:
# !rm model.tar.gz
# !aws s3 cp s3://sagemaker-us-east-1-835319576252/tensorflow-training-2021-01-28-01-19-50-987/output/model.tar.gz ./

In [53]:
# !rm -rf ./model
# !mkdir -p  ./model
# !tar -xvzf ./model.tar.gz -C model/

In [54]:
# !cp ./code/inference.py model/code/

In [55]:
# !cat model/code/inference.py